# How to use the API to pickle data

In [1]:
import os
import sys
from datetime import date
from pathlib import Path

curr_dir = Path(os.path.split(os.getcwd())[0])
metadata_dir = str(curr_dir)
if metadata_dir not in sys.path:
    sys.path.insert(0, metadata_dir)

import os
import glob
import time
import datetime
import numpy as np
import compress_pickle
import pandas as pd
from functools import partial
from multiprocessing import Pool
from bisect import bisect_left
from scipy.interpolate import interp1d
from scipy import signal
import matplotlib.pyplot as plt

## define source_path and destination_path

In [2]:
print("current working directory:",os.getcwd())

if os.path.isdir("/eos/project/m/ml-for-alarm-system/private/"):
    access2EOS = True
    print("Access to EOS possible")
    root_directory = "/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/"
else:
    access2EOS = False
    print("Access to EOS IMpossible")
    # search for a list of root_directory depending on the machine used to execute the notebook
    for directory in ["/home/thomas/cernboxML/private/CLIC_data_transfert/",
                      "?Christoph/CLIC_data_transfert/",
                      "?Andrea/CLIC_data_transfert/"]:
        if os.path.isdir(directory):
            root_directory = directory
print("Root directory:",root_directory)
 
# one week    
source_path = root_directory + "PSI1_XB2_Data/"

# 6 months
#source_path = root_directory + "CLIC_DATA_Xbox3/"

# don't forget the last / in the path
destination_path = source_path[0:-1]+"___pickle_only_TrendData_"+str(date.today())+"/" 

print("source_path of the data:",source_path)
print("destination_path of the pickle:",destination_path)

current working directory: /eos/home-l/lfischl/SWAN_projects/xbox3/notebooks
Access to EOS possible
Root directory: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/
source_path of the data: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/
destination_path of the pickle: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data___pickle_only_TrendData_2021-03-09/


## pickle

In [3]:
listOfTrendDataFiles = glob.glob(source_path+"Trend*.tdms")
listOfTrendDataFiles

['/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/TrendData_20180121.tdms',
 '/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/TrendData_20180122.tdms',
 '/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/TrendData_20180123.tdms',
 '/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/TrendData_20180124.tdms',
 '/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/TrendData_20180125.tdms',
 '/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/PSI1_XB2_Data/TrendData_20180126.tdms']

In [19]:
from nptdms import TdmsFile 

for dataFile in listOfTrendDataFiles:
    TrendData = TdmsFile.read(dataFile)
    for group in  TrendData.groups():
        group_name = group.name
        df = TrendData[group_name].as_dataframe()
        break

In [5]:
from nptdms import TdmsFile
trenddata = TdmsFile.open(listOfTrendDataFiles[0])

OrderedDict([('name', 'TrendData_20180121'), ('Version', 1)])

In [9]:

print(trenddata.properties)
print(trenddata.groups()[0].properties)
print(trenddata.groups()[0].channels()[1].properties)

OrderedDict([('name', 'TrendData_20180121'), ('Version', 1)])
OrderedDict()
OrderedDict([('wf_start_time', numpy.datetime64('1904-01-01T00:00:00.000000')), ('wf_start_offset', 0.0), ('wf_increment', 1.0), ('wf_samples', 1), ('NI_ChannelName', 'Loadside win'), ('NI_UnitDescription', 'mbar'), ('unit_string', 'mbar')])


In [20]:
np.set_printoptions(linewidth=160)

file = open("XB2_clms.txt", "w")  #change XB2 and XB3
file.writelines("%s\n" % str(clm) for clm in df.columns.values)
file.close()

In [21]:
f2 = open("XB2_clms.txt","r")
lines = [l[:-1] for l in f2.readlines()]
features2 = ["avg", "max", "min", "Pulse width"]
prop2 = ["Temp"]
for feature in features2:
    tmp_str = ""
    for l in lines:
        if feature in l:
            tmp_str += "\n" + l
    print(tmp_str+ "\n")
    
for prop in prop2:
    tmp_str = ""
    for l in lines:
        if prop in l:
            tmp_str += "\n" + l
    print(tmp_str + "\n")


PKI FT avg
PSI FT avg
PSR FT avg
PEI FT avg


PKI max
PSI max
PSR max
PEI max


BLM TIA min
BLM min
DC Down min
DC Up min




Klystron Flange Temp
Load Temp
PC Left Cavity Temp
PC Right Cavity Temp
Bunker WG Temp
Structure Input Temp



In [16]:
f3 = open("XB3_clms.txt","r")
lines = [l[:-1] for l in f3.readlines()]
features3 = ["mean", "max", "min", "pulse width"]
prop3 = ["Temp", "Preassure"]
for feature in features3:
    tmp_str = ""
    for l in lines:
        if feature in l:
            tmp_str += "\n" + l
    print(tmp_str + "\n")

for prop in prop3:
    tmp_str = ""
    for l in lines:
        if prop in l:
            tmp_str += "\n" + l
    print(tmp_str + "\n")


1KLYINA_ampcursor Blue mean 
1KLYINB_ampcursor Blue mean 
1PKIA_ampcursor Blue mean 
1PKIB_ampcursor Blue mean 
1PLIA_ampcursor Blue mean 
1PLIB_ampcursor Blue mean 
1PKRAcursor Blue mean 
1PKRBcursor Blue mean 
1PSI_ampcursor grey mean 
1PEI_ampcursor grey mean 
2KLYINA_ampcursor Blue mean 
2KLYINB_ampcursor Blue mean 
2PKIA_ampcursor Blue mean 
2PKIB_ampcursor Blue mean 
2PLIA_ampcursor Blue mean 
2PLIB_ampcursor Blue mean 
2PKRAcursor Blue mean 
2PKRBcursor Blue mean 
2PSI_ampcursor grey mean 
2PEI_ampcursor grey mean 


1KLYINA_amp max
1KLYINB_amp max
1PKIA_amp max
1PKIB_amp max
1PLIA_amp max
1PLIB_amp max
1PKRA max
1PKRB max
1PSI_amp max
1PEI_amp max
2KLYINA_amp max
2KLYINB_amp max
2PKIA_amp max
2PKIB_amp max
2PLIA_amp max
2PLIB_amp max
2PKRA max
2PKRB max
2PSI_amp max
2PEI_amp max




1KLYINA_amp pulse width
1KLYINB_amp pulse width
1PKIA_amp pulse width
1PKIB_amp pulse width
1PLIA_amp pulse width
1PLIB_amp pulse width
1PKRA pulse width
1PKRB pulse width
1PSI_amp pulse width
1PEI